## Deliverable 2. Create a Customer Travel Destinations Map.

In [58]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
import matplotlib.pyplot as plt
# Import the API key.
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.06,73,6,6.93,clear sky
1,1,Bambous Virieux,MU,-20.3428,57.7575,82.99,83,40,17.27,light rain
2,2,Egvekinot,RU,66.3167,-179.1667,-15.81,99,65,3.91,broken clouds
3,3,Vaini,TO,-21.2000,-175.2000,75.20,78,75,9.22,broken clouds
4,4,Cidreira,BR,-30.1811,-50.2056,70.61,83,100,14.23,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("Please input the min temp for your trip. "))
max_temp = float(input("Please input the max temp for your trip. "))

Please input the min temp for your trip. 80
Please input the max temp for your trip. 85


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.06,73,6,6.93,clear sky
1,1,Bambous Virieux,MU,-20.3428,57.7575,82.99,83,40,17.27,light rain
13,13,Souillac,MU,-20.5167,57.5167,80.60,83,40,17.27,scattered clouds
18,18,Mahibadhoo,MV,3.7833,72.9667,82.81,72,7,7.83,clear sky
32,32,Beloha,MG,-25.1667,45.0500,81.79,54,78,5.37,broken clouds


In [32]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID            0
City               0
Country            0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna()
pref_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.06,73,6,6.93,clear sky
1,1,Bambous Virieux,MU,-20.3428,57.7575,82.99,83,40,17.27,light rain
13,13,Souillac,MU,-20.5167,57.5167,80.60,83,40,17.27,scattered clouds
18,18,Mahibadhoo,MV,3.7833,72.9667,82.81,72,7,7.83,clear sky
32,32,Beloha,MG,-25.1667,45.0500,81.79,54,78,5.37,broken clouds
...,...,...,...,...,...,...,...,...,...,...
665,665,Shadegan,IR,30.6497,48.6647,84.20,28,40,11.50,scattered clouds
669,669,Marzuq,YE,14.4000,46.4667,83.52,8,49,2.95,scattered clouds
673,673,Yei,SS,4.0950,30.6779,84.16,49,90,3.78,overcast clouds
678,678,Goundam,ML,16.4145,-3.6708,80.29,13,0,9.04,clear sky


In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Kudahuvadhoo,MV,82.06,clear sky,2.6708,72.8944,
1,Bambous Virieux,MU,82.99,light rain,-20.3428,57.7575,
13,Souillac,MU,80.60,scattered clouds,-20.5167,57.5167,
18,Mahibadhoo,MV,82.81,clear sky,3.7833,72.9667,
32,Beloha,MG,81.79,broken clouds,-25.1667,45.0500,
50,Butaritari,KI,81.75,light rain,3.0707,172.7902,
52,Atuona,PF,80.92,few clouds,-9.8000,-139.0333,
53,Mahebourg,MU,80.60,light rain,-20.4081,57.7000,
67,Kavaratti,IN,83.55,clear sky,10.5669,72.6420,
74,Banda Aceh,ID,84.36,overcast clouds,5.5577,95.3222,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    print(f"Processing record {index}")
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("No Hotel .. Skipping")
        

Processing record 0
Processing record 1
Processing record 13
Processing record 18
Processing record 32
No Hotel .. Skipping
Processing record 50
Processing record 52
Processing record 53
Processing record 67
Processing record 74
Processing record 81
Processing record 91
Processing record 112
Processing record 121
Processing record 138
Processing record 139
Processing record 154
Processing record 160
Processing record 169
Processing record 195
Processing record 196
Processing record 198
Processing record 205
Processing record 206
Processing record 209
Processing record 217
Processing record 231
Processing record 237
Processing record 246
Processing record 260
Processing record 267
Processing record 286
Processing record 298
Processing record 302
Processing record 303
No Hotel .. Skipping
Processing record 312
Processing record 330
Processing record 342
No Hotel .. Skipping
Processing record 346
No Hotel .. Skipping
Processing record 358
Processing record 368
Processing record 370
Proces

In [45]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Kudahuvadhoo,MV,82.06,clear sky,2.6708,72.8944,Niyama Private Islands Maldives
1,Bambous Virieux,MU,82.99,light rain,-20.3428,57.7575,Casa Tia Villa
13,Souillac,MU,80.60,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
18,Mahibadhoo,MV,82.81,clear sky,3.7833,72.9667,Omadhoo Inn
32,Beloha,MG,81.79,broken clouds,-25.1667,45.0500,
...,...,...,...,...,...,...,...
665,Shadegan,IR,84.20,scattered clouds,30.6497,48.6647,مسافر خانه ابو ولید
669,Marzuq,YE,83.52,scattered clouds,14.4000,46.4667,
673,Yei,SS,84.16,overcast clouds,4.0950,30.6779,City Link Hotel
678,Goundam,ML,80.29,clear sky,16.4145,-3.6708,


In [52]:
# 7. Drop the rows where there is no Hotel Name.

# We are replacing the empty string to Null values
hotel_df.loc[(hotel_df["Hotel Name"]==""),"Hotel Name"]= np.nan

# We will now drop the 
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Kudahuvadhoo,MV,82.06,clear sky,2.6708,72.8944,Niyama Private Islands Maldives
1,Bambous Virieux,MU,82.99,light rain,-20.3428,57.7575,Casa Tia Villa
13,Souillac,MU,80.60,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
18,Mahibadhoo,MV,82.81,clear sky,3.7833,72.9667,Omadhoo Inn
50,Butaritari,KI,81.75,light rain,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
639,Biak,ID,80.55,overcast clouds,-0.9131,122.8766,Penginapan Kim
646,Luwuk,ID,82.36,overcast clouds,-0.9516,122.7875,Hotel Santika Luwuk
665,Shadegan,IR,84.20,scattered clouds,30.6497,48.6647,مسافر خانه ابو ولید
673,Yei,SS,84.16,overcast clouds,4.0950,30.6779,City Link Hotel


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPY_vacations.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weaher Description</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} *F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.


In [64]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))